# Introducción

Se ha notado que hay ciertos profesores que no consiguen acercar las adaptaciones necesarias a sus alumnos. El objetivo de nuestra tecnología sería predecir el nivel académico que va a desempeñar el alumno para estar más pendientes de aquellos alumnos que lo necesiten.

Para ello, se va a utilizar un dataset que contiene información sobre los alumnos y su rendimiento académico. Se va a realizar un análisis exploratorio de los datos para entender mejor la información que contienen y poder realizar un modelo predictivo que nos permita predecir el nivel académico de los alumnos.

# Dataset

## Columnas del dataset que vamos a usar
- `Gender`: Género del alumno
- `Age` : Edad del alumno
- `Study Hour/Day` : Horas de estudio al día
- `Learning mode` : Modalidad de aprendizaje
- `How many hour do you spent daily in social media?` : Horas diarias en redes sociales
- `Average attendance` : Asistencia media
- `With whom you are living?` : Con quién vive
- `What was your previous SGPA?` : Nota media anterior
- `What is your monthly family income?` : Nota media anterior

Lo primero que hemos hecho ha sido renombrar algunas de las columnas ya que no estaban muy bien redactadas. Además, hemos eliminado las columnas que no vamos a utilizar en nuestro análisis. 

Lo siguiente que hemos hecho ha sido eliminar las filas que contienen valores nulos.

Siguiendo esto hemos normalizado datos que estaban en formato de texto a formato numérico.

In [11]:
#Limpieza del dataset de los datos de los estudiantes.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import regex as re

#Cargamos el dataset

data = pd.read_csv('Students_Performance_data_set.csv')

pd.set_option('display.max_columns', None)

data.shape

#Renombramos las columnas que nos interesan

new_columns = {'Gender':'Gender', 'Age':'Age', 'Study Hour/day':'Study time', 'Learning mode':'Learning mode', 'How many hour do you spent daily in social media?':'Time on social media', 'Average attendance on class':'Average attendance', 'With whom you are living with?':'Cohabitants', 'What was your previous SGPA?':'Previous SGPA', 'What is your monthly family income?':'Family income'}

data.rename(columns=new_columns, inplace=True)

#Seleccionamos las columnas que nos interesan

columns_of_interest = ['Gender', 'Age', 'Study time', 'Learning mode', 'Time on social media', 'Average attendance', 'Cohabitants', 'Previous SGPA', 'Family income']

data = data[columns_of_interest]

#Eliminamos las filas con valores nulos
data = data.dropna()


#Estandarizamos los valores de distintas columnas

## Study time ------------------------------------------------------------------------------------------------------

### Usamos la función unique para ver los valores únicos de la columna y asi saber como estandarizarlos
# print(data['Study time'].unique())

def standarise_study_time(time): ## lo malo de esto es que pilla el que ha puesto 30 mins como si fueran 30 horas
    
    #Extract the numeric values from the string
    hours_numeric = re.findall(r'\d+', str(time))

    # Some of the fata is written in a range format, so we need to calculate the average
    if hours_numeric:
        if len(hours_numeric) == 1:
            return int(hours_numeric[0])
        else:
            return (int(hours_numeric[0]) + int(hours_numeric[1]))/2
    else:
        return None 
    
data['Study time test'] = data['Study time'].apply(standarise_study_time)

#display(data)
#print(data['Study time test'].isnull().sum())
#print(data['Study time'].unique())

# time on social media ------------------------------------------------------------------------------------------------------

print(data['Time on social media'].unique())

def standarise_social_time(time): ## tiene el mismo problema que el anterior en cuanto a lo de los minutos, ya lo mirare
    hours_numeric = re.findall(r'\d+', str(time))
    if hours_numeric:
        if len(hours_numeric) == 1:
            return int(hours_numeric[0])
        else:
            return (int(hours_numeric[0]) + int(hours_numeric[1]))/2
    else:
        return None

['2' '2 hours' 'Almost 1 hour' '3-4 hour' 'Less than 1' '1 hour'
 'more than 4 hours ' '3' 'Almost 2 hour' '5' '4' '1' 'Not define'
 '3 hours' '4-5 hour' '3-4hours' '1 hour ' '3-4h'
 'Lots of hour like half of a day' '4 hour' '2/3 hour' '2 hours ' '2-Jan'
 '2 hour' '2-3 hours' 'Yes' '3-Feb' 'Not fixed' 'Half hour' '6 hour'
 '2-3hr' '2 or 3 hours' '20min' '2-3h' 'Average 3 hours' '2 or 3'
 '6-7 hour' '5-Apr' '6-7 hours' '1 hr may be' '6 hours' '6' '6-7 hours '
 'Too much too.. Maybe 2 hour' '1hr' '15' '5 hours' '20-30mintues' '12'
 'Highest 2-3 hour' '3/4h' '30-40 Minutes' '30 min'
 '1 hour or sometimes less than 1 hour' 'Maximum time while needed'
 'Almost 3 hour' '4-Mar' '2H ' '5 hour' '10+' 'Less than 1 hour'
 'Around 3 hours' '2/3hour' '1-2hrs' '1hour per day' '3-Jan' '3hrs'
 '1-2 hours' '1 hours ' 'Around 2-3 hours ' 'More than 1 hour' '2-3 hour'
 '6-May' '2 hour ' '1-2 hour ' '1/2 hr' '2/3 hours' '3,4 hours'
 '3 to 4 hours' 'maybe 2 hour in total including youtube ' '0' '5h'
 '2ho